In [ ]:
# 실습에 필요한 라이브러리를 불러옵니다.
from torchtext import data, datasets
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# 텍스트 데이터 처리 방법을 정의합니다.
sentence_length = 500
TEXT = data.Field(batch_first=True, lower=True, fix_length=sentence_length)
LABEL = data.LabelField(batch_first=True)

In [ ]:
# IMDB 데이터 세트를 학습 세트와 테스트 세트로 나눠 불러옵니다.
train_datasets, test_datasets = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 21.6MB/s]


In [ ]:
# 학습 데이터의 첫 번째 데이터 샘플을 출력해봅니다.
print(vars(train_datasets[20000])['label'])

neg


In [ ]:
# 단어 집합을 정의합니다.
TEXT.build_vocab(train_datasets, max_size=10000)
LABEL.build_vocab(train_datasets)

# 단어 집합의 길이를 확인합니다.
print('number of total words: {}'.format(len(TEXT.vocab)))
print('number of total classes: {}'.format(len(LABEL.vocab)))

number of total words: 10002
number of total classes: 2


In [ ]:
# 데이터 이터레이터를 준비합니다.
train_iterator, test_iterator = data.BucketIterator.splits(
        (train_datasets, test_datasets),
        batch_size=32,
        device='cuda'
)

In [ ]:
# LSTM 모델 클래스를 정의합니다.
class LSTM(nn.Module):
  def __init__(self, vocab_size, sentence_length):
    super().__init__()
    # 문장의 길이를 저장합니다.
    self.sentence_length = sentence_length
    # 모델 구조를 정의합니다.
    self.embed = nn.Embedding(vocab_size, 16)
    self.cell = nn.LSTM(16, 16, batch_first=True)
    self.fc = nn.Linear(self.sentence_length * 16, 1)
    self.sigmoid = nn.Sigmoid()
 
  # 순전파를 정의합니다.
  def forward(self, X):
    out = self.embed(X)
    out, hidden_state = self.cell(out)
    out = out.contiguous()
    out = self.fc(out.view(-1, self.sentence_length * 16))
    out = self.sigmoid(out)
    return out

In [ ]:
# LSTM 모델 객체를 생성합니다.
vocab_size = len(TEXT.vocab)
model = LSTM(vocab_size, sentence_length).to('cuda')

# 이진 크로스 엔트로피(Binary Cross Entropy Error) 손실 함수 객체를 생성합니다.
criterion = nn.BCELoss().to('cuda')

# 아담 옵티마이저 객체를 생성합니다.
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, iterator):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  # 모델을 학습 모드로 설정합니다.
  model.train()

  # 배치 학습을 실행합니다.
  for batch in iterator:
    # 입력 데이터와 타깃을 준비합니다.
    X_batch, y_batch = batch.text, batch.label.float().view(-1, 1)
    # 기울기를 초기화합니다.
    optimizer.zero_grad()
    # 모델을 사용해 타깃을 예측합니다.
    hypothesis = model(X_batch)
    # 손실 함수로 오차를 계산합니다.
    loss = criterion(hypothesis, y_batch)        
    # 기울기를 계산합니다.
    loss.backward()
    # 경사 하강법으로 가중치를 수정합니다.
    optimizer.step()
    # 정확도를 계산합니다.
    acc = ((hypothesis >= 0.5) == y_batch).float().mean()
    # 현재 배치의 오차와 정확도를 저장합니다.
    epoch_loss += loss.item()
    epoch_acc += acc.item()

  # 현재 에포크의 오차와 정확도를 반환합니다.
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# 테스트 함수를 정의합니다.
def evaluate(model, criterion, optimizer, iterator):
  # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
  epoch_loss = 0
  epoch_acc = 0

  # 모델을 평가 모드로 설정합니다.
  model.eval()

  with torch.no_grad():
    # 배치 학습을 실행합니다.
    for batch in iterator:
      # 입력 데이터와 타깃을 준비합니다.
      X_batch, y_batch = batch.text, batch.label.float().view(-1, 1)
      # 모델을 사용해 타깃을 예측합니다.
      hypothesis = model(X_batch)
      # 손실 함수로 오차를 계산합니다.
      loss = criterion(hypothesis, y_batch)
      # 정확도를 계산합니다.
      acc = ((hypothesis >= 0.5) == y_batch).float().mean()
      # 현재 배치의 오차와 정확도를 저장합니다.
      epoch_loss += loss.item()
      epoch_acc += acc.item()

    # 현재 에포크의 오차와 정확도를 반환합니다.
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
# 10회에 걸쳐 모델을 학습합니다.
n_epoch = 10
for epoch in range(n_epoch):
  # 모델을 학습시킵니다.
  loss, acc = train(model, criterion, optimizer, train_iterator)

  # 모델을 평가합니다.
  test_loss, test_acc = evaluate(model, criterion, optimizer, test_iterator)

  # 현재 에포크의 학습 결과를 출력합니다.
  print('epoch: {}, loss: {:.3f}, acc: {:.2f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(
      epoch+1, loss, acc, test_loss, test_acc
  )) 

epoch: 1, loss: 0.637, acc: 0.61, test_loss: 0.542, test_acc: 0.728
epoch: 2, loss: 0.437, acc: 0.80, test_loss: 0.475, test_acc: 0.781
epoch: 3, loss: 0.325, acc: 0.86, test_loss: 0.407, test_acc: 0.821
epoch: 4, loss: 0.255, acc: 0.90, test_loss: 0.396, test_acc: 0.835
epoch: 5, loss: 0.203, acc: 0.92, test_loss: 0.400, test_acc: 0.838
epoch: 6, loss: 0.160, acc: 0.94, test_loss: 0.429, test_acc: 0.843
epoch: 7, loss: 0.128, acc: 0.95, test_loss: 0.450, test_acc: 0.844
epoch: 8, loss: 0.100, acc: 0.97, test_loss: 0.498, test_acc: 0.843
epoch: 9, loss: 0.073, acc: 0.98, test_loss: 0.579, test_acc: 0.842
epoch: 10, loss: 0.053, acc: 0.98, test_loss: 0.625, test_acc: 0.840
